In [23]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Checking the DATD Dataset

In [24]:
nRowsRead = None# specify 'None' if want to read whole file
# twitter_label_auto.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
DATD = pd.read_csv('DATD.csv', delimiter=',', nrows = nRowsRead)
nRow, nCol = DATD.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 2 columns


In [25]:
DATD.head(5)

,text,label
0,abis minum kopi perasaan mesti gelisah gatau k...,0
1,sumpah ya dari pagi rasanya gelisah terus,1
2,Catia hari sabtu.Presntasi Statin hari sabtu.....,1
3,tbtb gelisah 🤕,0
4,Memasuki jam jam rawan... Seliweran mulu di ga...,0


## Emoji Deletor

In [26]:
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F700-\U0001F77F"  # alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"# flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

## Setting up the Parameter

In [27]:
vocab_size = 4000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

## Cleaning the Data (Lowering case, deleting number, and deleting link)

In [28]:
import re
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
corpus = []
#http_pattern = "^(https?://)?/i"
for i in DATD['text']:
    data = i
    data = re.sub("https://t.co/\S*","",data)
    data = re.sub("\n","",data)
    data = re.sub("[0-9]","",data)
    data = deEmojify(data)
    #data = data.replace(http_pattern, "")
    corpus.append(data.lower())
    
len(corpus)

1000

### Corpus Berisi Data yang Sudah Bersih

In [30]:
corpus

['abis minum kopi perasaan mesti gelisah gatau knapa, tp tiap kali dibuat nonton vidio yg sedih sedih gampang bgt nangisnya, jd lebih lega ',
 'sumpah ya dari pagi rasanya gelisah terus',
 'catia hari sabtu.presntasi statin hari sabtu..on going :.laprak pske.laprak po.tugas tugas..dan sampe sekarsng belum clear latihan catia  produk hahahah..mantab, masih mode nyantuy dahal did dalem ya gelisah haha',
 'tbtb gelisah ',
 'memasuki jam jam rawan... seliweran mulu di gak di tonton sayang kalo ditonton gelisah ',
 'sering cemas buat hal sepele wajar ga si?',
 'aku bsa berobat sndiri, gak harus kamu yg obati, jd dia blg gini, aku sengaja menabrak kamu biar kamu bsa aku culik, ha buat apa kamu nyulik aku, untung nya apa, kamu kn sdah tau kalo aku sdah bersuami, kamu gak mikir apa keluarga aku pasti lagi nyarikin aku, mreka pasti gelisah',
 'bunda bunda busui, adakah yang pake retinol selama menyusui? huhu resah gelisah tahun depan udah  nee dan belum skincarean anti aging  gpp ga yah ke asi?

## Enkodisasi Text

In [31]:
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

{'<OOV>': 1, 'gelisah': 2, 'cemas': 3, 'yang': 4, 'dan': 5, 'aku': 6, 'di': 7, 'ada': 8, 'yg': 9, 'ini': 10, 'ga': 11, 'ya': 12, 'akan': 13, 'tidak': 14, 'hati': 15, 'apa': 16, 'itu': 17, 'orang': 18, 'rasa': 19, 'tapi': 20, 'tak': 21, 'gak': 22, 'aja': 23, 'kalo': 24, 'jadi': 25, 'gue': 26, 'lagi': 27, 'kita': 28, 'bisa': 29, 'kenapa': 30, 'hari': 31, 'kamu': 32, 'hal': 33, 'karena': 34, 'takut': 35, 'sendiri': 36, 'tenang': 37, 'bgt': 38, 'dengan': 39, 'untuk': 40, 'sama': 41, 'harap': 42, 'resah': 43, 'banget': 44, 'mau': 45, 'kau': 46, 'udah': 47, 'allah': 48, 'dia': 49, 'dari': 50, 'juga': 51, 'jangan': 52, 'hanya': 53, 'pada': 54, 'ke': 55, 'sudah': 56, 'menjadi': 57, 'tau': 58, 'pernah': 59, 'banyak': 60, 'diri': 61, 'nya': 62, 'tidur': 63, 'bila': 64, 'rasanya': 65, 'masa': 66, 'buat': 67, 'selalu': 68, 'harus': 69, 'paling': 70, 'sih': 71, 'sedih': 72, 'pasti': 73, 'semua': 74, 'tiba': 75, 'tuhan': 76, 'terus': 77, 'lain': 78, 'berlebihan': 79, 'suka': 80, 'pun': 81, 'kalau': 

## Tokenizing

Angka pada dekodisasi disusun jadi kalimat sesuai kalimat awal

In [34]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    input_sequences.append(token_list)
    
len(input_sequences)

1000

## Lihat Hasil Tokenizing

In [33]:
input_sequences

[[190,
  264,
  489,
  141,
  288,
  2,
  242,
  700,
  122,
  84,
  179,
  994,
  701,
  1561,
  9,
  72,
  72,
  490,
  38,
  1562,
  136,
  114,
  702],
 [243, 12, 50, 180, 65, 2, 77],
 [995,
  31,
  996,
  1563,
  1564,
  31,
  996,
  997,
  1565,
  998,
  1566,
  998,
  999,
  207,
  207,
  5,
  93,
  1567,
  128,
  1568,
  1569,
  995,
  1570,
  1571,
  1572,
  94,
  1000,
  1573,
  1574,
  1575,
  1576,
  12,
  2,
  570],
 [265, 2],
 [1001, 115, 115, 1002, 1577, 224, 7, 22, 7, 1578, 491, 24, 1579, 2],
 [158, 3, 67, 33, 1580, 1581, 11, 142],
 [6,
  703,
  1582,
  704,
  22,
  69,
  32,
  9,
  1583,
  136,
  49,
  1003,
  90,
  6,
  1004,
  1584,
  32,
  110,
  32,
  703,
  6,
  1585,
  1586,
  67,
  16,
  32,
  1587,
  6,
  1005,
  62,
  16,
  32,
  1588,
  1006,
  58,
  24,
  6,
  1006,
  1589,
  32,
  22,
  159,
  16,
  266,
  6,
  73,
  27,
  1590,
  6,
  1007,
  73,
  2],
 [705,
  705,
  1591,
  1592,
  4,
  571,
  1593,
  364,
  1594,
  706,
  43,
  2,
  289,
  123,
  47,
  

In [37]:
# pad sequences 
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
X = input_sequences
y = np.array(DATD['label'])

Kata terpanjang yang ada di dataset adalah 58 kata.

In [43]:
print(len(X[1]))
print(X)
print(y)

58
[[  0   0   0 ... 136 114 702]
 [  0   0   0 ...  65   2  77]
 [  0   0   0 ...  12   2 570]
 ...
 [  0   0   0 ... 218 186   1]
 [  0   0   0 ... 422   3 426]
 [  0   0   0 ... 984 128 988]]
[0 1 1 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0
 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 1
 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1
 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1
 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 0 1
 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1
 1 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 

## Sekarang udah Bisa dibagi datasetnya

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.8, random_state = 0)

## Mbuat Model (Contoh)

In [45]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [48]:
embedding_vecor_length = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_length))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           128000    
_________________________________________________________________
dense (Dense)                (None, 100, 1)            33        
Total params: 128,033
Trainable params: 128,033
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=75, batch_size=50)

Epoch 1/75
4/4 [==============================] - 13s 276ms/step - loss: 0.6923 - accuracy: 0.5438 - val_loss: 0.6917 - val_accuracy: 0.5440
Epoch 2/75
4/4 [==============================] - 0s 29ms/step - loss: 0.6893 - accuracy: 0.5844 - val_loss: 0.6906 - val_accuracy: 0.5506
Epoch 3/75
4/4 [==============================] - 0s 31ms/step - loss: 0.6868 - accuracy: 0.6104 - val_loss: 0.6895 - val_accuracy: 0.5543
Epoch 4/75
4/4 [==============================] - 0s 31ms/step - loss: 0.6826 - accuracy: 0.6472 - val_loss: 0.6886 - val_accuracy: 0.5579
Epoch 5/75
4/4 [==============================] - 0s 29ms/step - loss: 0.6781 - accuracy: 0.6720 - val_loss: 0.6877 - val_accuracy: 0.5609
Epoch 6/75
4/4 [==============================] - 0s 29ms/step - loss: 0.6728 - accuracy: 0.6911 - val_loss: 0.6871 - val_accuracy: 0.5638
Epoch 7/75
4/4 [==============================] - 0s 29ms/step - loss: 0.6738 - accuracy: 0.6675 - val_loss: 0.6866 - val_accuracy: 0.5636
Epoch 8/75
4/4 [=========